This notebook is forked from : https://www.kaggle.com/sudalairajkumar/simple-exploration-notebook-instacart/notebook

In this notebook, we will try and explore the basic information about the dataset given. The dataset for this competition is a relational set of files describing customers' orders over time. 

**Objective:** 

The goal of the competition is to predict which products will be in a user's next order. The dataset is anonymized and contains a sample of over 3 million grocery orders from more than 200,000 Instacart users.

For each user, 4 and 100 of their orders are given, with the sequence of products purchased in each order

Let us start by importing the necessary modules.

## Import modules

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()

%matplotlib inline

Let us list out the files that are present in this competition.!

In [ ]:
from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

Before we dive deep into the exploratory analysis, let us know a little more about the files given. To understand it better, let us first read all the files as dataframe objects and then look at the top few rows.

## Import data

In [ ]:
order_products_train_df = pd.read_csv("../input/order_products__train.csv")
order_products_prior_df = pd.read_csv("../input/order_products__prior.csv")
orders_df = pd.read_csv("../input/orders.csv")
products_df = pd.read_csv("../input/products.csv")
aisles_df = pd.read_csv("../input/aisles.csv")
departments_df = pd.read_csv("../input/departments.csv")

In [ ]:
orders_df.head()

In [ ]:
order_products_train_df.head()

In [ ]:
order_products_prior_df.head()

As we could see, orders.csv has all the information about the given order id like the user who has purchased the order, when was it purchased, days since prior order and so on.

The columns present in order_products_train and order_products_prior are same. Then what is the difference between these files.?

As mentioned earlier, in this dataset, 4 to 100 orders of a customer are given (we will look at this later) and we need to predict the products that will be re-ordered. So the last order of the user has been taken out and divided into train and test sets. All the prior order informations of the customer are present in order_products_prior file.  We can also note that there is a column in orders.csv file called eval_set which tells us as to which of the three datasets (prior, train or test) the given row goes to.

Order_products*csv file has more detailed information about the products that been bought in the given order along with the re-ordered status.

Let us first get the count of rows in each of the three sets.

In [ ]:
cnt_srs = orders_df.eval_set.value_counts()

plt.figure(figsize=(12,8))
sns.barplot(cnt_srs.index, cnt_srs.values, alpha=0.8, color=color[1])
plt.ylabel("# Occurences")
plt.xlabel("eval_set type")
plt.title("Count of rows in each dataset")
plt.xticks(rotation=45)
plt.show()

In [ ]:
def get_unique_count(x):
    return len(np.unique(x))

cnt_srs = orders_df.groupby("eval_set")["user_id"].aggregate(get_unique_count)
print(cnt_srs)

So there are 206,209 customers in total. Out of which, the last purchase of 131,209 customers are given as train set and we need to predict for the rest 75,000 customers. 

Now let us validate the claim that 4 to 100 orders of a customer are given. 

In [ ]:
cnt_srs = orders_df.groupby("user_id")["order_number"].aggregate(np.max)
cnt_srs = cnt_srs.reset_index()
cnt_srs = cnt_srs.order_number.value_counts()

plt.figure(figsize=(12,8))
sns.barplot(cnt_srs.index, cnt_srs.values, alpha=0.8)
plt.ylabel("# occurences")
plt.xlabel("Maximum order number")
plt.title("# Max order number")
plt.xticks(rotation=45)
plt.show()

So there are no orders less than 4 and is max capped at 100 as given in the data page. 

Now let us see how the ordering habit changes with day of week.

In [ ]:
plt.figure(figsize=(12,8))
sns.countplot(x="order_dow", data=orders_df, color=color[0])
plt.ylabel("Count")
plt.xlabel("Day of week")
plt.title("Frequency of order by week day")
plt.xticks(rotation=45)
plt.show()

Seems like 0 and 1 is Saturday and Sunday when the orders are high and low during Wednesday.

Now we shall see how the distribution is with respect to time of the day.

In [ ]:
plt.figure(figsize=(12,8))
sns.countplot(x="order_hour_of_day", data=orders_df, color=color[1])
plt.ylabel("# occurences")
plt.xlabel("Hour of day")
plt.title("Frequencies of order by hour of day")
plt.xticks(rotation=45)
plt.show()

So majority of the orders are made during day time. Now let us combine the day of week and hour of day to see the distribution.

In [ ]:
grouped_df = orders_df.groupby(["order_dow", "order_hour_of_day"])["order_number"]\
    .aggregate("count").reset_index()
grouped_df = grouped_df.pivot("order_dow", "order_hour_of_day", "order_number")

plt.figure(figsize=(12,8))
sns.heatmap(grouped_df)
plt.title("Frequency of day_of_week VS hour_of_day")
plt.show()

Seems Satuday evenings and Sunday mornings are the prime time for orders.

Now let us check the time interval between the orders.

In [ ]:
plt.figure(figsize=(12,8))
sns.countplot(x="days_since_prior_order", data=orders_df, color=color[3])
plt.ylabel("#")
plt.xlabel("days_since_prior_order")
plt.title("Frequency by days_since_prior_order")
plt.show()

Looks like customers order once in every week (check the peak at 7 days) or once in a month (peak at 30 days). We could also see smaller peaks at 14, 21 and 28 days (weekly intervals).

Since our objective is to figure out the re-orders, let us check out the re-order percentage in prior set and train set.

In [ ]:
order_products_prior_df["reordered"].sum() / order_products_prior_df.shape[0]


In [ ]:
order_products_train_df["reordered"].sum() / order_products_train_df.shape[0]


On an average, about 59% of the products in an order are re-ordered products.

**No re-ordered products:**

Now that we have seen 59% of the products are re-ordered, there will also be situations when none of the products are re-ordered. Let us check that now.

In [ ]:
def no_re_ordered_products_rate(df):
    grouped_df = df.groupby("order_id")["reordered"].aggregate("sum").reset_index()
    grouped_df["reordered"].loc[grouped_df["reordered"]>1] = 1
    return grouped_df.reordered.value_counts() / grouped_df.shape[0]

no_re_ordered_products_rate(order_products_prior_df)

In [ ]:
no_re_ordered_products_rate(order_products_train_df)

About 12% of the orders in prior set has no re-ordered items while in the train set it is 6.5%.

Now let us see the number of products bought in each order.

In [ ]:
grouped_df = order_products_train_df.groupby("order_id")["add_to_cart_order"].aggregate("max").reset_index()
cnt_srs = grouped_df["add_to_cart_order"].value_counts()

plt.figure(figsize=(12,8))
sns.barplot(cnt_srs.index, cnt_srs.values, alpha=0.8)
plt.ylabel("# Count")
plt.xlabel("# of products in the given order")
plt.show()

A right tailed distribution with the maximum value at 5.!

Before we explore the product details, let us look at the other three files as well. 

In [ ]:
products_df.head()

In [ ]:
aisles_df.head()

In [ ]:
departments_df.head()

Now let us merge these product details with the order_prior details.

In [ ]:
order_products_prior_df = pd.merge(order_products_prior_df, products_df, on='product_id', how='left')
order_products_prior_df = pd.merge(order_products_prior_df, aisles_df, on='aisle_id', how='left')
order_products_prior_df = pd.merge(order_products_prior_df, departments_df, on='department_id', how='left')
order_products_prior_df.head()

In [ ]:
cnt_srs = order_products_prior_df["product_name"].value_counts().reset_index()
cnt_srs = cnt_srs.head(20)
cnt_srs.columns = ["product_name", "frequency_count"]
cnt_srs

Wow. Most of them are organic products.! Also majority of them are fruits. 

Now let us look at the important aisles.

In [ ]:
cnt_srs = order_products_prior_df["aisle"].value_counts().head(20)
plt.figure(figsize=(12,8))
sns.barplot(cnt_srs.index, cnt_srs.values, alpha=0.8)
plt.ylabel("# count")
plt.xlabel("Aisle")
plt.title("Distribution of products by aisle")
plt.xticks(rotation=45)
plt.show()

The top two aisles are fresh fruits and fresh vegetables.! 

**Department Distribution:**

Let us now check the department wise distribution.

In [ ]:
temp_series = order_products_prior_df["department"].value_counts()
plt.figure(figsize=(8,8))
labels = (np.array(temp_series.index))
sizes = (np.array((temp_series/temp_series.sum())*100))

plt.pie(sizes, labels=labels, autopct="%1.1f%%", startangle=200)
plt.title("Distribution of products by department")
plt.show()

Produce is the largest department. Now let us check the reordered percentage of each department. 

**Department wise reorder ratio:**

In [ ]:
grouped_df = order_products_prior_df.groupby(["department"])["reordered"].aggregate("mean").reset_index()

plt.figure(figsize=(12,8))
sns.pointplot(grouped_df["department"].values, grouped_df["reordered"].values, alpha=0.8, color=color[2])
plt.ylabel("Reordered %")
plt.xlabel("Department")
plt.title("Reordered ratio by department")
plt.xticks(rotation=45)
plt.show()

Personal care has lowest reorder ratio and dairy eggs have highest reorder ratio.

**Aisle - Reorder ratio:**

In [ ]:
grouped_df = order_products_prior_df.groupby(["department_id", "aisle"])["reordered"]\
    .aggregate("mean").reset_index()

fig, ax = plt.subplots(figsize=(12,20))
ax.scatter(grouped_df.reordered.values, grouped_df.department_id.values)
for i, txt in enumerate(grouped_df.aisle.values):
    ax.annotate(txt, (grouped_df.reordered.values[i], grouped_df.department_id.values[i]), \
               rotation=45, ha="center", va="center", color="green")


plt.xlabel("Reorder ratio")
plt.ylabel("department_id")
plt.title("Reorder ratio by aisle and department")
plt.show()

**Add to Cart - Reorder ratio:**

Let us now explore the relationship between how order of adding the product to the cart affects the reorder ratio.

In [ ]:
order_products_prior_df["add_to_cart_order_mod"] = order_products_prior_df["add_to_cart_order"]\
    .copy()
order_products_prior_df["add_to_cart_order_mod"].loc[\
    order_products_prior_df["add_to_cart_order_mod"] >70] = 70
grouped_df = order_products_prior_df.groupby(["add_to_cart_order_mod"])["reordered"].aggregate("mean").reset_index()

plt.figure(figsize=(12,8))
sns.pointplot(grouped_df.add_to_cart_order_mod.values, grouped_df.reordered.values, alpha=0.8, color=color[3])   
plt.ylabel("Reorder ratio")
plt.xlabel("add_to_cart")
plt.title("Reorder ratio by add_to_cart")
plt.show()

**Looks like the products that are added to the cart initially are more likely to be reordered again compared to the ones added later.** This makes sense to me as well since we tend to first order all the products we used to buy frequently and then look out for the new products available. 

**Reorder ratio by Time based variables:**

Looks like reorder ratios are quite high during the early mornings compared to later half of the day.

**Hope it helped. Please leave your comments / suggestions.**